# Интеллектуальный анализ текстов

In [39]:
import scipy.spatial
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

## Расстояние Левенштейна 

In [3]:
fuzz.ratio("Игра", "Играть")/100

0.8

In [4]:
fuzz.ratio("Играющий", "Играть")/100

0.57

In [5]:
fuzz.ratio("Обедать", "Играть")/100

0.46

## Косинусное растояние
### Счетчик слов

In [6]:
sentences = [
    'привет друг',
    'привет привет',
    'как дела друг',
    'доброе утро',
    'что делаешь'
]
tokens = set(' '.join(sentences).split())
vectors = [[sentence.count(token) for token in tokens]
           for sentence in sentences]
print([(i, word) for i, word in enumerate(tokens)])
vectors

[(0, 'делаешь'), (1, 'утро'), (2, 'доброе'), (3, 'как'), (4, 'что'), (5, 'друг'), (6, 'привет'), (7, 'дела')]


[[0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 2, 0],
 [0, 0, 0, 1, 0, 1, 0, 1],
 [0, 1, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 1, 0, 0, 1]]

In [7]:
for i, sentence in enumerate(sentences):
    print(sentence)
    print(scipy.spatial.distance.cosine(vectors[0], vectors[i]))

привет друг
0.0
привет привет
0.29289321881345254
как дела друг
0.5917517095361369
доброе утро
1.0
что делаешь
1.0


### TF-IDF

In [78]:
vectorizer = TfidfVectorizer() #TfidfVectorizer(stop_words=['как', 'что'])
X = vectorizer.fit_transform(sentences)
len_vector = len(vectorizer.get_feature_names())
print([(i, word) for i, word in enumerate(vectorizer.get_feature_names())])
vectors = [list(i) for i in csr_matrix(X).toarray()]
vectors

[(0, 'дела'), (1, 'делаешь'), (2, 'доброе'), (3, 'друг'), (4, 'как'), (5, 'привет'), (6, 'утро'), (7, 'что')]


[[0.0, 0.0, 0.0, 0.7071067811865476, 0.0, 0.7071067811865476, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.6141889663426562,
  0.0,
  0.0,
  0.49552379079705033,
  0.6141889663426562,
  0.0,
  0.0,
  0.0],
 [0.0, 0.0, 0.7071067811865475, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0],
 [0.0, 0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475]]

In [79]:
for i, sentence in enumerate(sentences):
    print(sentence)
    print(scipy.spatial.distance.cosine(vectors[0], vectors[i]))

привет друг
0.0
привет привет
0.2928932188134524
как дела друг
0.6496117672881416
доброе утро
1.0
что делаешь
1.0


In [90]:
new_sentence = 'как дела товарищ'
print(csr_matrix(vectorizer.transform([new_sentence])).toarray()[0])

[0.70710678 0.         0.         0.         0.70710678 0.
 0.         0.        ]


## По наиболее схожему вопросу найти ответ

In [101]:
df = pd.read_excel('questions.xlsx')
df.head()

,question,answer
0,1. Что такое программирование?,Программирование – это процесс создания компью...
1,2. Как работает программный код?,"Код программы содержит инструкции, которые явл..."
2,3. Что такое отладка?,Отладка – это процесс поиска и устранения ошиб...
3,"4. Назовите типы ошибок, которые могут возникн...",Синтаксические ошибки.\nRuntime errors или оши...
4,5. Расскажите о синтаксических ошибках,"Синтаксическая ошибка возникает, когда в коде ..."


In [104]:
list(df['question'])[:15]

['1. Что такое программирование?',
 '2. Как работает программный код?',
 '3. Что такое отладка?',
 '4. Назовите типы ошибок, которые могут возникнуть в программе',
 '5. Расскажите о синтаксических ошибках',
 '6. Расскажите об ошибке времени выполнения',
 '7. Расскажите о логических ошибках',
 '8. Что такое блок-схема?',
 '9. Что такое алгоритм?',
 '10. Что по вашему значит «Поддерживать и обновлять программу»?',
 '11. Что такое переменные?',
 '12. Что такое зарезервированные слова в\xa0программировании?',
 '13. Что такое цикл?',
 '14. Назовите разные виды циклов',
 '15. Чем отличаются for и while?']

In [105]:
[item.replace('\xa0', ' ') for item in df['question']][:15]

['1. Что такое программирование?',
 '2. Как работает программный код?',
 '3. Что такое отладка?',
 '4. Назовите типы ошибок, которые могут возникнуть в программе',
 '5. Расскажите о синтаксических ошибках',
 '6. Расскажите об ошибке времени выполнения',
 '7. Расскажите о логических ошибках',
 '8. Что такое блок-схема?',
 '9. Что такое алгоритм?',
 '10. Что по вашему значит «Поддерживать и обновлять программу»?',
 '11. Что такое переменные?',
 '12. Что такое зарезервированные слова в программировании?',
 '13. Что такое цикл?',
 '14. Назовите разные виды циклов',
 '15. Чем отличаются for и while?']